In [ ]:
%%capture
!pip install git+https://github.com/optuna/optuna.git #installa la libreria optuna

In [ ]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn import linear_model

from keras import *
import tensorflow as tf
from keras.models import Sequential
from keras.layers.core import Dense, Dropout
from tensorflow.keras import regularizers
from keras.optimizers import Adam, RMSprop
from keras.backend import clear_session

import optuna
from optuna.samplers import RandomSampler
from google.colab import drive

In [1]:
train = pd.read_csv("/content/drive/MyDrive/AML project/train_manipulated.csv")

NameError: ignored

In [ ]:
## preparazione train e validation set

y_train = train['trip_duration'].values.reshape(-1, 1)

X_train = train.drop(columns = ['trip_duration']).values

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
y_train = scaler.fit(y_train).fit_transform(y_train)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25,
                                                  random_state = 1234)

In [ ]:
print(X_train.shape)
print(X_val.shape)

(1080324, 41)
(360109, 41)


In [ ]:
print(y_train.shape)
print(y_val.shape)

(1080324, 1)
(360109, 1)


In [ ]:
### inizia l'ottimizzazione

es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=10, restore_best_weights=True)

def objective(trial): 
  clear_session()
  initializer = tf.keras.initializers.GlorotUniform(seed=1234) #inizializiamo i pesi

  callbacks_list = [optuna.integration.KerasPruningCallback(trial, 'val_loss'), 
                      es]

  n_hidden = trial.suggest_categorical("#hidden", [3, 4])

  if n_hidden == 3:
    units_1 = trial.suggest_int("dense1", 16, 256, step = 16)
    units_2 = trial.suggest_int("dense2", 16, 128, step = 16)
    units_3 = trial.suggest_int("dense3", 16, 128, step = 16)
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2) 
    lambda_1 = trial.suggest_loguniform("lambda", 1e-4, 1e-1)
    activations = 'relu'

    model = Sequential()
    model.add(Dense(units_1, input_shape = (41,),
                  activation = activations, 
                  kernel_regularizer= regularizers.l2(lambda_1),
                  kernel_initializer=initializer))
    model.add(Dense(units_2,
                  activation = activations, 
                  kernel_regularizer= regularizers.l2(lambda_1),
                  kernel_initializer=initializer))
    model.add(Dense(units_3,
                  activation = activations, 
                  kernel_regularizer= regularizers.l2(lambda_1),
                  kernel_initializer=initializer))
    model.add(Dense(1,
                  activation = "linear", 
                  kernel_regularizer= regularizers.l2(lambda_1),
                  kernel_initializer=initializer))
  else:
    units_1 = trial.suggest_int("dense1", 16, 256, step = 16)
    units_2 = trial.suggest_int("dense2", 16, 128, step = 16)
    units_3 = trial.suggest_int("dense3", 16, 128, step = 16)
    units_4 = trial.suggest_int("dense4", 16, 64, step = 16)
    lr = trial.suggest_loguniform("lr", 1e-4, 1e-2) 
    lambda_1 = trial.suggest_loguniform("lambda", 1e-4, 1e-1)
    activations = 'relu'

    model = Sequential()
    model.add(Dense(units_1, input_shape = (41,),
                  activation = activations, 
                  kernel_regularizer= regularizers.l2(lambda_1),
                  kernel_initializer=initializer))
    model.add(Dense(units_2,
                  activation = activations, 
                  kernel_regularizer= regularizers.l2(lambda_1),
                  kernel_initializer=initializer))
    model.add(Dense(units_3,
                  activation = activations, 
                  kernel_regularizer= regularizers.l2(lambda_1),
                  kernel_initializer=initializer))
    model.add(Dense(units_4,
                  activation = activations, 
                  kernel_regularizer= regularizers.l2(lambda_1),
                  kernel_initializer=initializer))
    model.add(Dense(1,
                  activation = "linear", 
                  kernel_regularizer= regularizers.l2(lambda_1),
                  kernel_initializer=initializer))
    
  model.compile(loss = "mse", optimizer=Adam(lr=lr))

  model.fit(X_train, y_train, epochs = 25, 
            batch_size = 128, validation_data=(X_val, y_val),
            shuffle = True,
            callbacks = callbacks_list,
            verbose=False)
  
  score = model.evaluate(X_val, y_val, verbose=0)

  return score


In [ ]:
sampler = RandomSampler(123) #facciamo la random seaerch che almeno la sappiamo spiegare

study = optuna.create_study(study_name = "l2_reg_layers", direction="minimize", 
                            sampler = sampler, storage='sqlite:////content/drive/MyDrive/AML project/sqlite/optuna_l2_test2_layers.db', 
                            load_if_exists=True)

[I 2020-12-30 18:31:45,888] Using an existing study with name 'l2_reg_layers' instead of creating a new one.


In [ ]:
%%time
%%capture

study.optimize(objective, n_trials=50)

[I 2020-12-30 15:25:15,114] Trial 0 finished with value: 0.3242007791996002 and parameters: {'#hidden': 3, 'dense1': 224, 'dense2': 112, 'dense3': 48, 'lr': 0.0002842515929299163, 'lambda': 0.004507588967315677}. Best is trial 0 with value: 0.3242007791996002.
[I 2020-12-30 15:33:13,969] Trial 1 finished with value: 0.3365890681743622 and parameters: {'#hidden': 4, 'dense1': 176, 'dense2': 64, 'dense3': 32, 'dense4': 48, 'lr': 0.0006635119445083354, 'lambda': 0.005483816210985773}. Best is trial 0 with value: 0.3242007791996002.
[I 2020-12-30 15:38:37,775] Trial 2 finished with value: 0.317618191242218 and parameters: {'#hidden': 4, 'dense1': 160, 'dense2': 16, 'dense3': 112, 'dense4': 16, 'lr': 0.002871437810392841, 'lambda': 0.0020687915518060305}. Best is trial 2 with value: 0.317618191242218.
[I 2020-12-30 15:46:10,004] Trial 3 finished with value: 0.3150652050971985 and parameters: {'#hidden': 4, 'dense1': 80, 'dense2': 16, 'dense3': 16, 'dense4': 16, 'lr': 0.0018347801601630216, 

CPU times: user 2h 46min 41s, sys: 18min 14s, total: 3h 4min 56s
Wall time: 2h 9min 1s


In [ ]:
print('Best trial number: ', study.best_trial.number)

print("Number of finished trials: {}".format(len(study.trials)))
trial = study.best_trial

print("  Value: {}".format(trial.value))
print('Best parameters: \n', study.best_trial.params)

Best trial number:  32
Number of finished trials: 50
  Value: 0.29292359948158264
Best parameters: 
 {'#hidden': 3, 'dense1': 16, 'dense2': 16, 'dense3': 128, 'lambda': 0.00012096344724917057, 'lr': 0.0002505045069330552}


In [ ]:
df_trials = study.trials_dataframe()

In [ ]:
def maximizer(ll):

  best_ll = []
  
  best_ll.append(ll[0])

  smaller = ll[0]

  
  for i in range(len(ll)-1):
    
    if ll[i+1] < smaller : 

      best_ll.append(ll[i+1])

      smaller = ll[i+1]

    else:

      best_ll.append(smaller)

  
  return best_ll



df_trials['best_value'] = maximizer(df_trials['value'])
df_trials.head()

,number,value,datetime_start,datetime_complete,duration,params_#hidden,params_dense1,params_dense2,params_dense3,params_dense4,params_lambda,params_lr,state,best_value
0,0,0.324201,2020-12-30 15:17:53.635134,2020-12-30 15:25:15.057914,0 days 00:07:21.422780,3,224,112,48,NaN,0.004508,0.000284,COMPLETE,0.324201
1,1,0.336589,2020-12-30 15:25:15.131039,2020-12-30 15:33:13.922107,0 days 00:07:58.791068,4,176,64,32,48.0,0.005484,0.000664,COMPLETE,0.324201
2,2,0.317618,2020-12-30 15:33:13.982415,2020-12-30 15:38:37.722803,0 days 00:05:23.740388,4,160,16,112,16.0,0.002069,0.002871,COMPLETE,0.317618
3,3,0.315065,2020-12-30 15:38:37.790115,2020-12-30 15:46:09.955229,0 days 00:07:32.165114,4,80,16,16,16.0,0.002093,0.001835,COMPLETE,0.315065
4,4,0.298336,2020-12-30 15:46:10.017386,2020-12-30 15:53:30.593182,0 days 00:07:20.575796,3,128,48,80,NaN,0.000782,0.000717,COMPLETE,0.298336
